In [ ]:
# Set and create directories for the TAO Toolkit experiment
import os

os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

!mkdir $PROJECT_DIR
!mkdir $DATA_DIR
!mkdir $MODELS_DIR

In [ ]:
import os
os.environ['NGC_DIR']='/dli/task/ngc_assets'
os.environ['CLI']='ngccli_linux.zip'

# Remove previous versions of NGC CLI, copy, and install NGC CLI
!rm -r $NGC_DIR/ngccli/*
!cp /dli/task/$CLI $NGC_DIR/ngccli/$CLI
!unzip -u "$NGC_DIR/ngccli/$CLI" \
       -d $NGC_DIR/ngccli/
!rm $NGC_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("NGC_DIR", ""), os.getenv("PATH", ""))

In [ ]:
# List all available models
!ngc registry model list nvidia/tao/* --column name --column repository --column application

In [ ]:
# Download the unpruned pre-trained model from NGC
!ngc registry model download-version nvidia/tao/trafficcamnet:unpruned_v1.0 \
    --dest $MODELS_DIR

In [ ]:
# Download the pruned pre-trained model from NGC
!ngc registry model download-version nvidia/tao/trafficcamnet:pruned_v1.0 \
    --dest $MODELS_DIR

In [ ]:
# Check if models have been downloaded into directory
!ls -rlt $MODELS_DIR

In [ ]:
# View the video
from IPython.display import Video

Video("data/126_206-A0-3_raw.mp4", width=720)

In [ ]:
# Preview the annotation
!cat $SOURCE_DATA_DIR/126_206-A0-3_json_sample.txt

In [ ]:
# Load the .csv into a DataFrame
import ast
import pandas as pd

annotated_frames=pd.read_csv('data/annotation.csv', converters={2:ast.literal_eval})
print("Length of the full DF object:", len(annotated_frames))
annotated_frames.head()

In [ ]:
# Check how many rows per frame_no
annotated_frames.groupby('frame_no').size()

In [ ]:
# Filter out annotations that do not have car inside the bbox
filtered_frames=annotated_frames[annotated_frames["outside"] == 0]
print("Length of the filtered DF object:", len(filtered_frames))
filtered_frames.head()

In [ ]:
# Plot frames that include moving cars
import matplotlib.pyplot as plt
import numpy as np

frames_list=list(filtered_frames['frame_no'].unique())
frame_existance=np.zeros(annotated_frames['frame_no'].max()+1)
for i in frames_list:
    frame_existance[int(i)]=1
y_pos=np.arange(len(frame_existance))
fig, ax=plt.subplots(figsize=(18, 3))
plt.bar(y_pos, frame_existance, align='center', alpha=0.5)
plt.title('Frame Indices that Include Moving Cars')
plt.yticks([])
plt.show()

In [ ]:
# Define function to extract images and generate an annotated video
import cv2
colors = [(255, 255, 0), (255, 0, 255), (0, 255, 255), (0, 0, 255), (255, 0, 0), (0, 255, 0), (0, 0, 0), (255, 100, 0), (100, 255, 0), (100, 0, 255), (255, 0, 100)]

def save_images(video_path, image_folder, frames_list, annotated_frames,  video_out_folder, fps=10):
    # Create image folder if it doesn't exist
    if not os.path.exists(image_folder):
        print("Creating images folder")
        os.makedirs(image_folder)
    
    # Create directory for output video
    if not os.path.exists(video_out_folder):
        print("Creating video out folder")
        os.makedirs(video_out_folder)
    
    # Start reading input video
    input_video=cv2.VideoCapture(video_path)
    
    # cv2.VideoCapture().read() returns true if it has a next frame
    retVal, im=input_video.read()
    size=im.shape[1], im.shape[0]
    fourcc=cv2.VideoWriter_fourcc(*'mp4v') 
    
    # Start writing output video
    output_video=cv2.VideoWriter('{}/annotated_video.mp4'.format(video_out_folder), fourcc, fps, size)

    frameCount=0
    i=1
    
    # While has next frame
    while retVal:
        print("\rProcessing frame no: {}".format(frameCount), end='', flush=True)
        
        # If current frame is in the list of annotated frames, draw bounding box(es) and include in the output video
        if frameCount in frames_list:
            print("\rSaving frame no: {}, index: {} out of {}".format(frameCount, i, len(frames_list)), end='')
            cv2.imwrite(os.path.join(image_folder, '{}.png'.format(frameCount)), im)
            i+=1
            frame_items=annotated_frames[annotated_frames["frame_no"]==int(frameCount)]
            for index, box in frame_items.iterrows():
                xmin, ymin, xmax, ymax = box["xmin"], box["ymin"], box["xmax"], box["ymax"]
                xmin2, ymin2, xmax2, ymax2 = box["crop"][0], box["crop"][1], box["crop"][2], box["crop"][3]
                cv2.rectangle(im, (xmin, ymin), (xmax, ymax), colors[0], 1)
                cv2.rectangle(im, (int(xmin2), int(ymin2)), (int(xmax2), int(ymax2)), colors[1], 1)
            output_video.write(im)

        # Read next frame
        retVal, im=input_video.read()
        frameCount+=1

    input_video.release()
    output_video.release()
    return size        

In [ ]:
# Extract images and generate an annotated video
save_images('{}/126_206-A0-3_raw.mp4'.format(os.environ['SOURCE_DATA_DIR']), 
            '{}/{}'.format('{}/training'.format(os.environ['DATA_DIR']), 'images'),
            frames_list,
            filtered_frames,
            '{}/{}'.format(os.environ['DATA_DIR'], 'video_out'))

In [ ]:
# Convert the video to a format that is compatible with Jupyter Lab
!ffmpeg -i tao_project/data/video_out/annotated_video.mp4 tao_project/data/video_out/annotated_video_conv.mp4 -y -loglevel quiet

In [ ]:
# View the annotated output video
Video('tao_project/data/video_out/annotated_video_conv.mp4', width=720)

In [ ]:
# Generate labels in KITTI format
label_folder='{}/training/labels'.format(os.environ['DATA_DIR'])
if not os.path.exists(label_folder):
    print("Creating labels folder")
    os.makedirs(label_folder)
for frame in sorted(frames_list): 
    current_frame=filtered_frames[filtered_frames['frame_no']==frame]
    with open('{}/{}.txt'.format(label_folder, frame), 'w') as f: 
        for i, box in current_frame.iterrows(): 
            print('Writing for frame {}'.format(frame), end='\r')
            f.write("Car 0 0 0 {} {} {} {} 0 0 0 0 0 0 0\n".format(box['xmin'], box['ymin'], box['xmax'], box['ymax']))

In [ ]:
# Preview sample KITTI format labels
!cat $DATA_DIR/training/labels/20.txt

In [ ]:
# View the spec file
!cat $SPEC_FILES_DIR/kitti_config.txt

In [ ]:
# View dataset_convert usage
!detectnet_v2 dataset_convert --help

In [ ]:
# Create directory for TFRecords and delete existing files if they exist
!mkdir -p $DATA_DIR/tfrecords && rm -rf $DATA_DIR/tfrecords/*

!detectnet_v2 dataset_convert -d $SPEC_FILES_DIR/kitti_config.txt \
                              -o $DATA_DIR/tfrecords/kitti_trainval/kitti_trainval

In [ ]:
# Check the shards that have been created
!ls -rlt $DATA_DIR/tfrecords/kitti_trainval/